In [5]:
import os
import numpy as np
import cv2
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, precision_score, roc_auc_score,f1_score,roc_curve
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.applications import VGG16, VGG19
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import optimizers
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image
from keras.saving import save_model

/Users/swetha/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
pip install sklearn

Defaulting to user installation because normal site-packages is not writeable
    ERROR: Command errored out with exit status 1:
     command: /Library/Developer/CommandLineTools/usr/bin/python3 -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/mm/9dc76b_j2358wnq6r5qmm6w40000gn/T/pip-install-9cfnv9c4/sklearn_b31876e36dfa47ad8bd65a82423c134e/setup.py'"'"'; __file__='"'"'/private/var/folders/mm/9dc76b_j2358wnq6r5qmm6w40000gn/T/pip-install-9cfnv9c4/sklearn_b31876e36dfa47ad8bd65a82423c134e/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /private/var/folders/mm/9dc76b_j2358wnq6r5qmm6w40000gn/T/pip-pip-egg-info-t8k6o97o
         cwd: /private/var/folders/mm/9dc76b_j2358wnq6r5qmm6w40000gn/T/pip-install-9cfnv9c4/sklea

In [1]:
pip install opencv-python-headless


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 35.4 MB 5.6 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
train_dir = '//Users//swetha//Documents//DLipynb files//cancer//train'
test_dir = '//Users//swetha//Documents//DLipynb files//cancer//test'

img_width, img_height = 224, 224

defining the batch size

In [ ]:
batch_size = 32


Data augumentation: data augumentation is used to increase the no of images and sometimes we can zoom, shear, rotate so the images are trained more precisely.

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1./255,         
    rotation_range=20,       
    width_shift_range=0.2,   
    height_shift_range=0.2, 
    shear_range=0.2,         
    zoom_range=0.2,          
    horizontal_flip=True,    
    vertical_flip=True,   
    validation_split = 0.2,
    fill_mode='nearest'      
)

normalizing the images

In [8]:
test_datagen = ImageDataGenerator(rescale=1./255)


In [10]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary'  
)

Found 2637 images belonging to 2 classes.


In [12]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary'  
)

Found 660 images belonging to 2 classes.


checking the labels:

In [13]:
print("Class labels for train set:", train_generator.class_indices)
print("Class labels for test set:", test_generator.class_indices)

Class labels for train set: {'benign': 0, 'malignant': 1}
Class labels for test set: {'benign': 0, 'malignant': 1}


building vgg model

In [14]:
def vggModel(input_shape, num_classes):
    
    # Loading pre-trained model 
    
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)  # Uncomment for VGG19
    
    for layer in base_model.layers:
        layer.trainable = False

    model = Sequential([
        base_model,
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='sigmoid')
    ])

    return model

defining cnn model

In [15]:
def cnnModel(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(256, (3, 3), activation='relu'), # relu introduces non linearity
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='sigmoid')
    ])

    return model

In [16]:
input_shape = (224, 224, 3)  
num_classes = 1 

In [17]:
cnn_model = cnnModel(input_shape, num_classes)


/Users/swetha/Library/Python/3.9/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [19]:
vgg19_model = vggModel(input_shape, num_classes)


80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 62s 1us/step


In [20]:
vgg19_model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg19 (Functional)              │ ?                      │    20,024,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,024,384 (76.39 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 20,024,384 (76.39 MB)

In [21]:
cnn_model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 24, 24, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 12, 12, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36864)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    18,874,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,526,465 (74.49 MB)

 Trainable params: 19,526,465 (74.49 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
optimizer = optimizers.Adam(learning_rate=0.001)


In [23]:
vgg19_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


In [24]:
cnn_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


In [25]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [26]:
buildVgg19 = vgg19_model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=5,
    callbacks=[early_stopping]
)

Epoch 1/5


/Users/swetha/Library/Python/3.9/lib/python/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


83/83 ━━━━━━━━━━━━━━━━━━━━ 44322s 540s/step - accuracy: 0.5780 - loss: 1.6151 - val_accuracy: 0.7833 - val_loss: 0.4476
Epoch 2/5
83/83 ━━━━━━━━━━━━━━━━━━━━ 1251s 15s/step - accuracy: 0.7154 - loss: 0.5562 - val_accuracy: 0.8167 - val_loss: 0.4199
Epoch 3/5
83/83 ━━━━━━━━━━━━━━━━━━━━ 1684s 20s/step - accuracy: 0.7081 - loss: 0.5908 - val_accuracy: 0.8076 - val_loss: 0.4787
Epoch 4/5
83/83 ━━━━━━━━━━━━━━━━━━━━ 1254s 15s/step - accuracy: 0.7283 - loss: 0.5232 - val_accuracy: 0.7909 - val_loss: 0.4874
Epoch 5/5
83/83 ━━━━━━━━━━━━━━━━━━━━ 2338s 28s/step - accuracy: 0.7226 - loss: 0.5303 - val_accuracy: 0.7697 - val_loss: 0.5131


In [ ]:
buildCNN = cnn_model.fit(
    train_generator,
    validation_data = test_generator,
    epochs = 5,
    # callbacks = [early_stopping]
)

In [ ]:
vgg19_test_loss, vgg19_test_acc = vgg19_model.evaluate(test_generator)
custom_cnn_test_loss, custom_cnn_test_acc = cnn_model.evaluate(test_generator)

print("VGG19 Test Loss:", vgg19_test_loss)
print("VGG19 Test Accuracy:", vgg19_test_acc)
print("\nCustom CNN Test Loss:", custom_cnn_test_loss)
print("Custom CNN Test Accuracy:", custom_cnn_test_acc)

In [ ]:
vgg19_predictions = vgg19_model.predict(test_generator)
vgg19_predicted_labels = (vgg19_predictions > 0.5).astype(int) 

In [ ]:

vgg19_accuracy = accuracy_score(test_generator.labels, vgg19_predicted_labels)
vgg19_precision = precision_score(test_generator.labels, vgg19_predicted_labels)
vgg19_recall = recall_score(test_generator.labels, vgg19_predicted_labels)
vgg19_f1 = f1_score(test_generator.labels, vgg19_predicted_labels)

In [ ]:
print("Metrics for VGG19 Model:")
print("Accuracy:", vgg19_accuracy)
print("Precision:", vgg19_precision)
print("Recall:", vgg19_recall)
print("F1-score:", vgg19_f1)
print()

In [ ]:
custom_cnn_predictions = cnn_model.predict(test_generator)
custom_cnn_predicted_labels = (custom_cnn_predictions > 0.5).astype(int) 

In [ ]:
custom_cnn_accuracy = accuracy_score(test_generator.labels, custom_cnn_predicted_labels)
custom_cnn_precision = precision_score(test_generator.labels, custom_cnn_predicted_labels)
custom_cnn_recall = recall_score(test_generator.labels, custom_cnn_predicted_labels)
custom_cnn_f1 = f1_score(test_generator.labels, custom_cnn_predicted_labels)

In [ ]:
print("Metrics for Custom CNN Model:")
print("Accuracy:", custom_cnn_accuracy)
print("Precision:", custom_cnn_precision)
print("Recall:", custom_cnn_recall)
print("F1-score:", custom_cnn_f1)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(buildVgg19.history['loss'], label='VGG19 Training Loss')
plt.plot(buildCNN.history['loss'], label='Custom CNN Training Loss')
plt.title('Training Loss Curves')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(buildVgg19.history['accuracy'], label='VGG19 Training Accuracy')
plt.plot(buildCNN.history['accuracy'], label='Custom CNN Training Accuracy')
plt.title('Training Accuracy Curves')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
test_labels = test_generator.classes


In [ ]:
fpr_vgg19, tpr_vgg19, _ = roc_curve(test_labels, vgg19_predictions)
fpr_custom_cnn, tpr_custom_cnn, _ = roc_curve(test_labels, custom_cnn_predictions)

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(fpr_vgg19, tpr_vgg19, label='VGG19')
plt.plot(fpr_custom_cnn, tpr_custom_cnn, label='Custom CNN')
plt.plot([0, 1], [0, 1], 'k--', label='Random Guess')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves')
plt.legend()
plt.grid(True)
plt.show()